In [1]:
import pandas as pd
import numpy as np

The probatility values from the Win predictive model

In [2]:
def get_pbvalues(teamName):

    if teamName == 'GT':

        p_0 = 0.29030
        p_1 = 0.29030 + 0.42742
        p_2 = 0.29030 + 0.42742 + 0.06759
        p_3 = 0.29030 + 0.42742 + 0.06759 + 0.00244
        p_4 = 0.29030 + 0.42742 + 0.06759 + 0.00244 + 0.12192
        p_6 = 0.29030 + 0.42742 + 0.06759 + 0.00244 + 0.12192 + 0.04716
        p_w = 1

    elif teamName == 'CSK':
        p_0 = 0.32438
        p_1 = 0.32438 + 0.40581
        p_2 = 0.32438 + 0.40581 + 0.06554
        p_3 = 0.32438 + 0.40581 + 0.06554 + 0.00372
        p_4 = 0.32438 + 0.40581 + 0.06554 + 0.00372 + 0.10861
        p_6 = 0.32438 + 0.40581 + 0.06554 + 0.00372 + 0.10861 + 0.05025
        p_w = 1

    return p_0, p_1, p_2, p_3, p_4, p_6, p_w

In [3]:
def predict_runs(target, current_score, current_wickets, current_overs):

    # pb values of both teams
    i1p_0, i1p_1, i1p_2, i1p_3, i1p_4, i1p_6, i1p_w = get_pbvalues('GT')
    i2p_0, i2p_1, i2p_2, i2p_3, i2p_4, i2p_6, i2p_w = get_pbvalues('CSK')

    pred_runs = current_score
    pred_wks = current_wickets
    leftover_balls = 120 - current_overs*6

    for i in range(leftover_balls):
        r_value = np.random.random()

        if r_value <= i2p_0:
            pred_runs += 0
        elif r_value <= i2p_1:
            pred_runs += 1
        elif r_value <= i2p_2:
            pred_runs += 2
        elif r_value <= i2p_3:
            pred_runs += 3
        elif r_value <= i2p_4:
            pred_runs += 4
        elif r_value <= i2p_6:
            pred_runs += 6
        else:
            pred_runs += 0
            pred_wks += 1
            if pred_wks == 10:
                break
        if pred_runs > target:
            break
    return pred_runs

In [4]:
predict_runs(192, 101, 2, 9)

184

In [5]:
def get_win(pred_runs, target):
    if pred_runs > target:
        return 'win'
    elif pred_runs == target:
        return 'tie'
    else:
        return 'lose'

In [6]:
target = 192

current_score = 101
current_wickets = 1
current_overs = 11

iter_count = 100

runs_ls = []
results_ls = []

win_count = 0
tie_count = 0
lose_count = 0

for i in range(iter_count):
    pred_runs = predict_runs(target, current_score, current_wickets, current_overs)
    runs_ls.append(pred_runs)
    result_pred = get_win(pred_runs, target)
    results_ls.append(result_pred)

    if result_pred == 'win':
        win_count += 1
    elif result_pred == 'tie':
        tie_count += 1
    else:
        lose_count +=1

win_count, tie_count, lose_count

(2, 0, 98)

In [7]:
def find_runs(current_score, target, current_wickets, at_overs):
    runs_ls = []
    results_ls = []

    req_runs = []
    win_ls = []

    for i in range(current_score, target + 1):
        win_count = 0
        tie_count = 0
        lose_count = 0

        for j in range(100):
            pred_runs = predict_runs(target, i, current_wickets, at_overs)
            runs_ls.append(pred_runs)
            result_pred = get_win(pred_runs, target)
            results_ls.append(result_pred)

            if result_pred == 'win':
                win_count += 1
            elif result_pred == 'tie':
                tie_count += 1
            else:
                lose_count +=1

            win_ls.append(win_count)
            req_runs.append(i)

    required_runs = current_score
    for i in range(len(req_runs)):
        if win_ls[i] >= 50:
            required_runs = req_runs[i]
            break

    return required_runs

In [8]:
find_runs(101, 171, 1, 11)

102

In [9]:
def find_wickets(current_score, target, current_wickets, at_overs):
    req_runs = find_runs(current_score, target, current_wickets, at_overs)

    runs_ls = []
    results_ls = []

    req_wks = []
    win_ls = []

    for i in range(current_wickets, 10):
        win_count = 0
        tie_count = 0
        lose_count = 0

        for j in range(100):
            pred_runs = predict_runs(target, current_score, i, at_overs)
            runs_ls.append(pred_runs)
            result_pred = get_win(pred_runs, target)
            results_ls.append(result_pred)

            if result_pred == 'win':
                win_count += 1
            elif result_pred == 'tie':
                tie_count += 1
            else:
                lose_count +=1

        win_ls.append(win_count)
        req_wks.append(i)

    req_wicket_value = current_wickets

    for i in range(len(req_wks)):
        if (win_ls[i] < 45)  :
            req_wicket_value = req_wks[i]
            break

    return req_wicket_value

In [10]:
find_wickets(101, 171, 1, 11)

3

Interactive Chart

In [11]:
%matplotlib inline
from ipywidgets import interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

def find_runs_wickets(current_wks, at_overs, target_score):
    plt.figure(figsize = (16, 6))
    x = np.array(list(range(21)))
    req_value = find_runs(101, target_score, current_wks, at_overs)
    req_wk_value = find_wickets(101, target_score, current_wks, at_overs)

    if at_overs == 11:
        req_value = 101
        req_wk_value = 1

    y = np.array([req_value for i in range(21)])

    plt.scatter(at_overs, req_value, s = 1200, color = 'red')
    plt.axhline(target_score, ls = '--', color = 'blue')
    plt.text( 1, target_score + 10, 'Target Score :' + str(target_score) , color = 'darkblue', fontsize = 13)
    plt.text( at_overs, req_value, str(req_value) + '/' + str(req_wk_value), color = 'white', fontsize = 12,  horizontalalignment='center', verticalalignment='center')
    plt.text(at_overs, req_value - 30, 'CSK has to be at ' + str(req_value) + '/' +  str(req_wk_value) + ' after ' + str(at_overs) + ' ov', horizontalalignment='center')
    plt.ylim(50, target_score + 50)
    plt.xticks(x)
    plt.title('Where should CSK be?', fontsize = 20)
    plt.xlabel('Overs')
    plt.ylabel('Score')
    plt.show()

In [12]:
print('current_score = CSK: 101/1 (11 overs)')
print('')

interactive_plot = interactive(find_runs_wickets, current_wks = widgets.IntSlider(min=1, max=10, step=1, value=1),  at_overs=widgets.IntSlider(min=10, max=20, step=1, value=11), target_score = widgets.IntSlider(min=0, max=250, step=1, value=171))
output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot

current_score = CSK: 101/1 (11 overs)



interactive(children=(IntSlider(value=1, description='current_wks', max=10, min=1), IntSlider(value=11, descri…